In [9]:
!pip install streamlit pandas pyttsx3 SpeechRecognition requests python-dotenv google-api-python-client openpyxl

In [23]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2

In [53]:
code = """
import streamlit as st
import pandas as pd
import sqlite3
import speech_recognition as sr
import pyttsx3
import os
import threading
from dotenv import load_dotenv
import requests
from googleapiclient.discovery import build

# Load environment variables
load_dotenv()

# API Keys from environment
google_api_key = os.getenv("your_google_api_key")
youtube_api_key = os.getenv("your_youtube_api_key")
huggingface_key = os.getenv("your_hugginggace _api_key")

# SQLite DB setup
conn = sqlite3.connect('chat_history.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS conversations (role TEXT, content TEXT)''')
conn.commit()

# Text-to-Speech safely using threading
def speak(text):
    def _speak():
        engine = pyttsx3.init()
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak).start()

# Save and fetch chat history
def save_conversation(role, content):
    c.execute('INSERT INTO conversations (role, content) VALUES (?, ?)', (role, content))
    conn.commit()

def fetch_conversation_history():
    c.execute('SELECT role, content FROM conversations ORDER BY ROWID DESC LIMIT 10')
    return [{"role": r, "content": c} for r, c in reversed(c.fetchall())]

# Hugging Face Text Generation (AI replies)
# Function to generate a response using Hugging Face
def generate_response(user_input):
    headers = {"Authorization": f"Bearer {"your_huggingface_api_key"}"}
    payload = {
        "inputs": user_input,
        "parameters": {
            "temperature": 0.7,  # Controls the randomness of the output
            "max_new_tokens": 100,  # Limits the length of the response
            "return_full_text": False  # Only return the generated text
        }
    }

    # Send POST request to Hugging Face API
    response = requests.post(
        "https://api-inference.huggingface.co/models/distilbert-base-uncased-finetuned-sst-2-english",  # You can replace this with any model you like
        headers=headers,
        json=payload
    )

    if response.status_code == 200:
        result = response.json()
        return result[0]['generated_text']
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return "Sorry, I couldn't generate a reply right now."

# Hugging Face Sentiment
def huggingface_sentiment(text):
    headers = {"Authorization": f"Bearer {"your_huggingface_api_key"}"}
    response = requests.post(
        "https://api-inference.huggingface.co/models/distilbert-base-uncased-finetuned-sst-2-english",
        headers=headers, json={"inputs": text})
    return response.json()

# YouTube Search
def search_youtube(query):
    youtube = build('youtube', 'v3', developerKey="your_youtube_api_key")
    request = youtube.search().list(q=query, part='snippet', type='video', maxResults=3)
    response = request.execute()
    return response['items']

# --- Streamlit UI ---
st.set_page_config(page_title="DS/ML Chatbot (No OpenAI)", layout="wide")
st.title("🤖 Data Science & ML Assistant (No OpenAI)")
st.write("Ask questions, analyze sentiment, and find relevant YouTube videos!")

# File Upload
uploaded_file = st.file_uploader("📁 Upload CSV or Excel file", type=["csv", "xlsx"])
if uploaded_file:
    df = pd.read_csv(uploaded_file) if uploaded_file.name.endswith(".csv") else pd.read_excel(uploaded_file)
    st.write(df)

# Chat history
messages = fetch_conversation_history()
for msg in messages:
    st.markdown(f"**{'🧑 You' if msg['role'] == 'user' else '🤖 Bot'}:** {msg['content']}")

# Voice or text input
user_input = None
if st.button("🎙️ Speak"):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        st.write("Listening...")
        audio = recognizer.listen(source)
    try:
        user_input = recognizer.recognize_google(audio)
        st.write(f"You said: {user_input}")
    except Exception as e:
        st.error(f"Voice input error: {e}")
else:
    user_input = st.text_input("💬 Ask your question")

# Process user query
if user_input:
    save_conversation("user", user_input)

    with st.spinner("Thinking..."):
        # AI response
        bot_reply = generate_response(user_input)
        st.markdown(f"**🤖 Bot:** {bot_reply}")
        speak(bot_reply)
        save_conversation("assistant", bot_reply)

        # YouTube video results
        st.subheader("🎥 Related YouTube Videos")
        yt_results = search_youtube(user_input)
        for vid in yt_results:
            video_url = f"https://www.youtube.com/watch?v={vid['id']['videoId']}"
            st.markdown(f"- [{vid['snippet']['title']}]({video_url})")

        # Sentiment analysis
        st.subheader("🧠 Sentiment Analysis")
        sentiment = huggingface_sentiment(user_input)
        st.json(sentiment)

"""

In [51]:
with open("Ds_ml_chatbotai.py", "w", encoding="utf-8") as f:
    f.write(code)

print("File saved as Ds_ml_chatbotai.py ✅")

File saved as Ds_ml_chatbotai.py ✅


In [55]:
# how can run this code in tour terminal 
#in terminal do 
# import pyaudio for speak to search
# cd " name where your file saved"
# dir to cheack file
# streamlit run your file saving name.py
# import any thing according to you in your terminal window to run this AI code